# Source: Soybase 
Data for 20,087 G. max and G. soja accessions genotyped with 42,509 SNPs (Wm82.a2)

scikit-allel is a Python package intended to enable exploratory analysis of large-scale genetic variation

# import the libraries

In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
sns.set_style('ticks')
sns.set_context('notebook')
import h5py  # hdf5 file 
import allel;  # # import scikit-allel
print('scikit-allel', allel.__version__) # check which version is installed

scikit-allel 1.3.5


# read_vcf()

In [2]:
# import scikit-allel
import allel
# check which version is installed
print(allel.__version__)

1.3.5


In [3]:
callset = allel.read_vcf('soysnp50k_wm82.a2_41317.vcf.gz')

In [4]:
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

# vcf_to_dataframe()

The vcf_to_dataframe() function extracts all data except samples and genotyping calls from a VCF and loads into a df.

In [5]:
df1 = allel.vcf_to_dataframe('soysnp50k_wm82.a2_41317.vcf.gz')
df1.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,QUAL,FILTER_PASS
0,Chr01,24952,ss715578788,A,G,NaN,NaN,NaN,False
1,Chr01,26003,ss715578818,C,T,NaN,NaN,NaN,False
2,Chr01,29671,ss715578923,A,G,NaN,NaN,NaN,False
3,Chr01,30712,ss715578960,G,A,NaN,NaN,NaN,False
4,Chr01,37018,ss715579193,C,T,NaN,NaN,NaN,False


# samples array (header line from VCF file)

In [6]:
# sampels array
samples=callset['samples']
print(samples.size)
samples

20087


array(['PI86046', 'PI90208', 'PI219698', ..., 'PI587906', 'PI587946',
       'PI603516'], dtype=object)

In [7]:
# convert samples array to dataframe
df2 = pd.DataFrame(data = samples)
#df2.head()

In [8]:
# transpose samples dataframe
samples_df=df2.T
samples_df

,0,1,2,3,4,5,6,7,8,9,...,20077,20078,20079,20080,20081,20082,20083,20084,20085,20086
0,PI86046,PI90208,PI219698,PI253651A,PI347550A,PI398807,PI408055A,PI408069,PI408169A,PI408169B,...,PI574480B,PI578360,PI578362,PI639693,PI657626,PI634759,PI423967,PI587906,PI587946,PI603516


In [9]:
# column headers of dataframe
samples_df.columns = samples_df.iloc[0]
samples_df

,PI86046,PI90208,PI219698,PI253651A,PI347550A,PI398807,PI408055A,PI408069,PI408169A,PI408169B,...,PI574480B,PI578360,PI578362,PI639693,PI657626,PI634759,PI423967,PI587906,PI587946,PI603516
0,PI86046,PI90208,PI219698,PI253651A,PI347550A,PI398807,PI408055A,PI408069,PI408169A,PI408169B,...,PI574480B,PI578360,PI578362,PI639693,PI657626,PI634759,PI423967,PI587906,PI587946,PI603516


# genotype array from VCF file

-1 to indicate a missing value; 0=reference, 1=alternate allele

In [10]:
geno=callset['calldata/GT'] 
geno[:1]

array([[[ 1,  1],
        [ 0,  0],
        [-1, -1],
        ...,
        [ 0,  0],
        [ 0,  0],
        [ 0,  1]]], dtype=int8)

In [11]:
# Genotype array
genotypes=allel.GenotypeArray(geno)
genotypes

<GenotypeArray shape=(42195, 20087, 2) dtype=int8>
1/1 0/0 ./. 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/1
1/1 0/0 ./. 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/1
1/1 0/0 0/1 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/1
...
0/0 0/0 0/0 0/0 1/1 ... 0/0 0/0 0/0 1/1 ./.
0/0 1/1 1/1 1/1 1/1 ... 1/1 0/0 1/1 0/1 1/1
0/0 0/0 0/0 0/0 1/1 ... 0/0 0/0 0/0 ./. 0/0

### Reshape genotype array to view it as haplotypes by dropping the ploidy dimension

seems to_haplotypes() generates hapotype arrays with doubled the number of samples (from 20087 to 40174); each value (1/1) is split in to two columns (1 and 1). 

In [12]:
haps=genotypes.to_haplotypes() 
haps

<HaplotypeArray shape=(42195, 40174) dtype=int8>
1 1 0 0 . ... 0 0 0 0 1
1 1 0 0 . ... 0 0 0 0 1
1 1 0 0 0 ... 0 0 0 0 1
...
0 0 0 0 0 ... 0 1 1 . .
0 0 1 1 1 ... 1 0 1 1 1
0 0 0 0 0 ... 0 . . 0 0

In [13]:
#print('dtype:', haps.dtype)
#print('shape:',haps.shape)
#print('No of assays/rows:',haps.n_variants)
#print('No of samples/columns:', haps.n_haplotypes)

In [14]:
# Allele calls for a single variant at all haplotypes/samples
#haps[1]

In [15]:
#A single haplotype/sample can be obtained by indexing the second dimension
#haps[:, 1]

In [16]:
# allele call for a single haplotype/sample at a single variant 
#haps[1, 0]

In [17]:
# Reshape a haplotype array as diploid genotypes
#haps.to_genotypes(ploidy=2) # i.e reverting back to GenotypeArray

In [18]:
# create dataframe of haplotypes by transposing the array (with out transposing, memory is too much and getting error)
haps_df=pd.DataFrame(haps.T)
haps_df.head()  # so markers turned as columns; GE's as rows

,0,1,2,3,4,5,6,7,8,9,...,42185,42186,42187,42188,42189,42190,42191,42192,42193,42194
0,1,1,1,0,0,0,1,0,1,1,...,0,0,0,1,1,1,1,0,0,0
1,1,1,1,0,0,0,1,0,1,1,...,0,0,0,1,1,1,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0
4,-1,-1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0


In [19]:
haps_df.shape

(40174, 42195)

In [ ]:
haps_df.to_csv('input_data.csv.gz')

In [ ]:
df_iterator = pd.read_csv('input_data.csv.gz', chunksize=1000, compression='gzip')

for i, df_chunk in enumerate(df_iterator):

    df_chunk.groupby(np.arange(len(df_chunk))//2).sum()
    #do_something(df_chunk)
    
    # Set writing mode to append after first chunk
    mode = 'w' if i == 0 else 'a'
    
    # Add header if it is the first chunk
    header = i == 0

    df_chunk.to_csv(
        "dst_data.csv.gz",
        index=False,  # Skip index column
        header=header, 
        mode=mode,
        compression='gzip')

In [ ]:
dataset=pd.read_csv("dst_data.csv.gz")
dataset.head()

In [ ]:
## because of memory issue, I am reading chunks of 1000 rows from haps_df a

In [ ]:
from more_itertools import sliced
CHUNK_SIZE = 1000

index_slices = sliced(range(len(haps_df)), CHUNK_SIZE)

#data=[]
for index_slice in index_slices:
    chunk = haps_df.iloc[index_slice] # your dataframe chunk ready for use
    result = chunk.groupby(np.arange(len(chunk))//2).sum() 
    #data.append(result)
    result.to_csv('modified.csv')
    


In [ ]:
chunked_df = pd.DataFrame(data)
chunked_df.head()

In [ ]:
haps_df1=haps_df.head()
haps_df1

In [ ]:
haps_df1.shape

In [ ]:
result = haps_df1.groupby(np.arange(len(haps_df1))//2).sum()
result

In [ ]:
combined_df=pd.concat([df1, samples_df])

# vcf_to_hdf5()

For large datasets, vcf to hdf5 is good; HDF5 file stored on disk.

In [ ]:
#vcf_path='soysnp50k_wm82.a2_41317.vcf.gz'

In [ ]:
allel.vcf_to_hdf5('soysnp50k_wm82.a2_41317.vcf.gz', 'soysnp50k_wm82.a2.h5', fields='*', overwrite=True)

In [ ]:
df1.shape

In [ ]:
callset_fn = 'soysnp50k_wm82.a2.h5'
callset = h5py.File(callset_fn, mode='r')
callset

In [ ]:
callset.keys()

In [ ]:
chrom = callset['variants/CHROM']
chrom[1:5]

In [ ]:
pos = callset['variants/POS']
pos

In [ ]:
# load all items into NumPy array
pos[1:3]

In [ ]:
# load genotype calls into memory for second to fourth variants, all samples
gt = callset['calldata/GT']
gt


In [ ]:
genotypes=allel.GenotypeArray(gt)
genotypes

In [ ]:
geno_array=genotypes.reshape(genotypes.shape[0], genotypes.shape[1], genotypes.shape[2])
geno_array.shape

In [ ]:
import pandas as pd
df = pd.DataFrame(geno_array)
#df.to_csv('50k_geno_calls.csv')
df.head()

# pick a chromosome to work

In [ ]:
chrom = 'scaffold_759'

# Visualize variant density

Plot shows how many SNPs are there and how they are distributed along the chromosome

# Filtering

Drop any polymorphic SNP with rate of missing & het alleles >0.1 among the 19,648 soybean and wild soybean accessions. The het allele calls in the remaining loci were set as missing in the subsequent analysis


# Similarity analysis
Genetic similarity between pairs of genotypes among the 18,480 cultivated and among the 1168 wild accessions was calculated as the ratio of
the number of identical SNP allele calls and the total number of SNPs for
which allele calls were made for the pair

# Cluster analysis
Pair-wise distance among the accessions of 806 wild and 5396 landrace
soybeans was obtained based on the allelic dissimilarity of the 42,509
SNPs; the neighbor-joining tree was constructed

# LD analysis
LD was analyzed within the wild, landrace, and N. Am. cultivar
populations with 806, 5396, and 562 accessions, respectively. Only
the SNPs with minor allele frequency $5% were included for LD calculation and construction of haplotype blocks. Calculation of pairwise
LD (r2
) among SNPs and identification of haplotype blocks was based
upon SNPs within 1-Mb windows using the software PLINK (Purcell
et al. 2007).

In [ ]:
# Dendrogam of wild and landrace genotypes from different countries.